## Importing required libraries

In [78]:
!pip install biopython

In [79]:
from Bio import SeqIO
from Bio.Seq import Seq 
from Bio.SeqRecord import SeqRecord 
import gzip 
import glob 
import string 
import itertools

## Extra resources

In [80]:
punctuation = string.punctuation  ## using this package to get a list of possible punctuations in utf-8 format. 

In [82]:
numbers_list = [0,1,2,3,4,5,6,7,8,9]

## Starting code to evaluate primer amplification

In [93]:
forward_primer ='-GCDGCHGCNATGCGTTAYAC-3'
reverse_primer = '-ACAAGMTCWGCKATTTTTTC-3'

In [94]:
source_path = 'data/' 

In [95]:
def decompress_gzip(gzip_path, output_path):
    with gzip.open(gzip_path, 'rt') as f_in:
        with open(output_path, 'wt') as f_out:
            f_out.write(f_in.read())

In [96]:
files = glob.glob(source_path + '/*.gz')

In [97]:
for file in files:
    if file.endswith('.gz'):
        gzip_path =  file
        fasta_path = file.replace('.gz', '') 

        decompress_gzip(gzip_path, fasta_path)

In [98]:
def treating_primer(primer): 
    for nucleotide in primer: 
            if nucleotide in punctuation or nucleotide in str(numbers_list): 
                primer = str(primer).replace(nucleotide, '')
    return primer


forward_primer = treating_primer(forward_primer) 
reverse_primer = treating_primer(reverse_primer) 

In [99]:
print(forward_primer, reverse_primer)

GCDGCHGCNATGCGTTAYAC ACAAGMTCWGCKATTTTTTC


In [100]:
def generate_primer_combinations(primer):
  
    replacements = {
        'R': ['G', 'A'],
        'Y': ['C', 'T'],
        'M': ['A', 'C'],
        'K': ['G', 'T'],
        'S': ['G', 'C'],
        'W': ['A', 'T'],
        'B': ['C', 'G', 'T'],
        'D': ['A', 'G', 'T'],
        'H': ['A', 'C', 'T'],
        'V': ['A', 'C', 'G'],
        'N': ['A', 'C', 'G', 'T']
    }

   
    parts = []
    
  
    for char in primer:
        if char in replacements:
            parts.append(replacements[char])
        else:
            parts.append(char)
    
    
    combinations = [''.join(p) for p in itertools.product(*parts)]
    
    return combinations

In [101]:
treated_forward_primer = generate_primer_combinations(forward_primer) 
treated_reverse_primer = generate_primer_combinations(reverse_primer)

In [102]:
reverse_complement_primer = []

for item in treated_reverse_primer: 
    my_seq = Seq(item)
    reverse_complement_primer.append(my_seq.reverse_complement())

In [103]:
filepath = glob.glob(source_path + '*.fna')

def evaluating_amplification(filepath):
    amplified_files = [] 
    field = []
    for file in filepath:
        for seq_record in SeqIO.parse(file, "fasta"):
            sequence = seq_record.seq
            
            for forward_primer in treated_forward_primer:
                for reverse_primer in reverse_complement_primer:
                    if forward_primer in sequence and reverse_primer in sequence: 
                        position_fw = sequence.find(forward_primer) 
                        position_rv = sequence.find(reverse_primer) + len(reverse_primer) 
                        field.append(sequence[position_fw: position_rv])
                        print(field)
                        
                        amplified_files.append(file)
                        print(file, seq_record.id, f'amplificação dupla -  com o foward primer: {forward_primer} e amplificação com o reverso complementar {reverse_primer}') 
             
    amplified_files = list(dict.fromkeys(amplified_files)) 
    return {'amplified_files': amplified_files,
            'field': field}

In [104]:
results = evaluating_amplification(filepath) 

[Seq('GCGGCCGCGATGCGTTACACAGAAGCGAGAATGTCAAAAATCGCAATGGAAATC...TGT')]
data/genome_05.fna contig_0 amplificação dupla -  com o foward primer: GCGGCCGCGATGCGTTACAC e amplificação com o reverso complementar GAAAAAATCGCAGATCTTGT


In [105]:
def save_sequence_as_fasta(sequence, output_path, record_id="amplified_sequence"):
    seq_record = SeqRecord(Seq(sequence), id=record_id, description="")
    SeqIO.write(seq_record, output_path, "fasta")
    print(f"Sequência salva em {output_path}") 


save_sequence_as_fasta(results['field'][0], 'amplified_sequence.fasta', record_id='amplified_sequence')

Sequência salva em amplified_sequence.fasta


# Blast

In [106]:
def blast_sequence(fasta_file):
    with open(fasta_file) as fasta:
        result_handle = NCBIWWW.qblast("blastn", "nt", fasta.read())
        return result_handle

In [107]:
def parse_blast_result(result_handle):
    blast_record = NCBIXML.read(result_handle)
    
    best_hit = None
    for alignment in blast_record.alignments:
        for hsp in alignment.hsps:
            if best_hit is None or hsp.expect < best_hit['e_value']:
                best_hit = {
                    'title': alignment.title,
                    'length': alignment.length, 
                    'start': hsp.sbjct_start,
                    'end': hsp.sbjct_end,
                    'e_value': hsp.expect,
                    'sequence': hsp.sbjct
                }
                
    return best_hit


In [108]:
result_handle = blast_sequence("amplified_sequence.fasta")
best_hit = parse_blast_result(result_handle)

if best_hit:
    print('****Best Hit****')
    print('Title:', best_hit['title'])
    print('Length:', best_hit['length'])
    print('Start position:', best_hit['start'])
    print('End position:', best_hit['end'])
    print('E-value:', best_hit['e_value'])
    print('Sequence:', best_hit['sequence'])
else:
    print('No significant hits found.')


****Best Hit****
Title: gi|2769841181|gb|CP160221.1| Bacillus velezensis strain AP215 chromosome, complete genome
Length: 3929792
Start position: 6943
End position: 7433
E-value: 0.0
Sequence: GCGGCCGCGATGCGTTACACAGAAGCGAGAATGTCAAAAATCGCAATGGAAATCCTCCGGGACATTACGAAAGATACGATTGATTATCAAGATAACTATGACGGCGCAGAAAGAGAACCTGTCGTCATGCCTTCGAGATTTCCGAATCTGCTCGTAAACGGAGCTGCCGGTATTGCGGTCGGAATGGCGACAAATATTCCTCCGCATCAGCTTGGGGAAGTCATTGAAGGCGTGCTTGCCGTAAGTGAGAATCCTGAGATTACAAACCAGGAGCTGATGGAATACATCCCGGGCCCGGATTTTCCGACTGCAGGTCAGATTTTGGGCCGGAGCGGCATCCGCAAGGCATATGAATCCGGACGGGGATCCATTACGATCCGGGCTAAGGCTGAAATCGAAGAGACATCATCGGGAAAAGAAAGAATTATTGTCACAGAACTTCCTTATCAGGTGAACAAAGCGAGATTAATTGAAAAAATCGCAGATCTTGT
